In [2]:
import pandas as pd
import os
from nltk.corpus import stopwords
import nltk.data
import logging
import time
import numpy as np  # Make sure that numpy is imported
from gensim.models import Word2Vec
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier

from KaggleWord2VecUtility import KaggleWord2VecUtility
import json
import cPickle as pickle
import re
from operator import itemgetter

In [60]:
# whocalled has 171660 with comments among 555146 records
#scrapy_data = '/home/fujun/phone-data/whocalled.json'
# callercomplaints has 184245 unique phones with comments among 188130 records
data_root = '/home/fujunl/phone-scraping-data/'
scrapy_data = data_root + 'callercomplaints.json'
phone_stat = dict()
caller_type = dict()
year_min = 2008
time_slots = 12*(2016 - year_min + 1)
default_type_dist = {u'Debt Collector': 0, u'Unknown': 0, u'Prank Caller': 0, u'Political Call': 0, u'Telemarketer': 0}
with open(scrapy_data) as f:
    data = json.load(f)
    for record in data:
        phone_id = record['phone_id'].split('-')[0]
        if phone_id not in phone_stat:
            phone_stat[phone_id] = {'n_comments':0, 'time_dist':np.zeros(time_slots), 'type_dist':dict(default_type_dist)}
            
        for comment in record['phone_comments']:
            if comment:
                phone_stat[phone_id]['n_comments'] += 1
                                   
                time_str = re.search('[0-9]+/[0-9]+/[0-9]+', comment['time']).group()
                call_time = datetime.strptime(time_str, '%m/%d/%Y')
                slot_id = 12*(call_time.year - year_min) + call_time.month - 1
                phone_stat[phone_id]['time_dist'][slot_id] += 1
                
                #year_min = min(year_min, call_time.year)
                tmp_type = comment['caller_type']
                if tmp_type in caller_type:
                    caller_type[tmp_type] += 1
                else:
                    caller_type[tmp_type] = 1
                phone_stat[phone_id]['type_dist'][tmp_type] += 1

#with open('callercomplaints_stat.p', 'wb') as fp:
    #pickle.dump(phone_stat, fp)

In [58]:
# load data and do analysis
with open('callercomplaints_stat.p', 'rb') as fp:
    phone_stat = pickle.load(fp)
    
phone_ncomments = [(phone_id, phone_stat[phone_id]['n_comments']) for phone_id in phone_stat.keys()]
sorted_phones = sorted(phone_ncomments, key=itemgetter(1), reverse=True)
print len(sorted_phones)
for stat in sorted_phones[:1]:
    print stat[0], stat[1]
    print phone_stat[stat[0]]['n_comments']

184245
5125025384 4141
4141


In [90]:
# whocalled has 171660 with comments among 555146 records
#scrapy_data = '/home/fujun/phone-data/whocalled.json'
# callercomplaints has 184245 unique phones with comments among 188130 records
data_root = '/home/fujunl/phone-scraping-data/'
scrapy_data = data_root + 'whocalled.json'
phone_stat = dict()
year_min = 2005
time_slots = 12*(2016 - year_min + 1)
location_stat = dict()
with open(scrapy_data) as f:
    data = json.load(f)
    for record in data:
        if not record['comments']:
            continue
        phone_id = record['phone']
        if phone_id not in phone_stat:
            phone_stat[phone_id] = {'n_comments':0, 'time_dist':np.zeros(time_slots)}
        for comment in record['comments']:
            if comment:
                if comment['location']:
                    location = comment['location'][0].strip().lower()
                    loc_index = 0
                    for m in re.finditer('[0-9]+', location):
                        loc_index = max(loc_index, m.span()[1])
                    location = location[loc_index:].strip()
                    
                    #if location1 != location:
                        #print location, location1
                        
                    if location in location_stat:
                        location_stat[location] = location_stat[location]+1
                    else:
                        location_stat[location] = 1
                    
                phone_stat[phone_id]['n_comments'] += 1
                #print str(comment['time'])
                call_time = datetime.strptime(comment['time'][0], '%Y-%m-%d %H:%M:%S')
                slot_id = 12*(call_time.year - year_min) + call_time.month - 1
                phone_stat[phone_id]['time_dist'][slot_id] += 1

with open('whocalled_stat.p', 'wb') as fp:
    pickle.dump(phone_stat, fp)
    
with open('whocalled_location_stat.p', 'wb') as fp:
    pickle.dump(location_stat, fp)

In [75]:
phone_ncomments = [(phone_id, phone_stat[phone_id]['n_comments']) for phone_id in phone_stat.keys()]
sorted_phones = sorted(phone_ncomments, key=itemgetter(1), reverse=True)
print len(sorted_phones)
for stat in sorted_phones[:10]:
    print stat[0], stat[1]
    #print phone_stat[stat[0]]['n_comments']

171660
0000000000 1315
8009681875 895
8592121501 884
8004461022 826
2013738371 785
8009184224 783
5132176631 783
3239271550 669
6136883625 645
8006790336 643


In [99]:
print location_stat['ghana']

11


In [139]:
from geopy.geocoders import Nominatim
geolocator = Nominatim()
location_geo = dict()
sidemap = {'la':'Los Angeles,California', 'nt canada':'Northwest Territories,canada', 
        'virgin islands, u.s.':'virgin islands', 'il':'Illinois'}
for addr in location_stat.keys():
    try:
        lat_long = geolocator.geocode(addr)
    except GeocoderTimedOut as e:
        print("Error: geocode failed on input {} with message {}".format(addr, e.msg))
    
    if not lat_long:
        print 'geo for {} not found'.format(addr)
        if addr in mymap:
            lat_long = geolocator.geocode(mymap[addr])
            if not lat_long:
                print 'geo for {} NOT found even using {}'.format(addr,mymap[addr])
            else:
                print 'geo for {} found using {}'.format(addr,mymap[addr])
        else:
            print 'I don not know how to find {}'.format(addr)
    
    #print '{}:{}'.format(addr, str(lat_long))
    if lat_long:
        location_geo[addr] = {'latitude':lat_long.latitude, 
                              'longitude':lat_long.longitude, 'n_called':location_stat[addr]}

print len(location_geo), len(location_stat)
with open('location_geo.p', 'wb') as fp:
    pickle.dump(location_geo, fp)

geo for la not found
geo for la found using Los Angeles,California
geo for virgin islands, u.s. not found
geo for virgin islands, u.s. found using virgin islands
geo for nt canada not found
geo for nt canada found using Northwest Territories,canada
geo for satellite provider not found
I don not know how to find satellite provider
geo for il not found
geo for il found using Illinois
224 225


In [16]:
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
with open('location_geo.p', 'rb') as fp:
    location_geo = pickle.load(fp)

fig = plt.figure()
themap = Basemap(projection='gall', resolution = 'l', area_thresh = 100000.0)
themap.drawcoastlines()
themap.drawcountries()
themap.fillcontinents(color = 'gainsboro')
themap.drawmapboundary(fill_color='steelblue')

lons = [location['longitude'] for location in location_geo.values()]
lats = [location['latitude'] for location in location_geo.values()]
called = np.array([location['n_called'] for location in location_geo.values()])
mark_sz = 3*((called // 10000) + 1)

x, y = themap(lons, lats)
for i in range(len(x)):
    themap.plot(x[i], y[i], 'o', color='Red', markersize=mark_sz[i])

plt.title('Geographical Distribution of Scams Received')
plt.show()

fig.savefig('call.png')

In [8]:

print np.max(mark_sz), np.min(mark_sz),np.mean(mark_sz)

6 1 1.12053571429
